### Chains with multiple inputs

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

llm = ChatOpenAI()

prompt_template = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input="a parrot")

{'input': 'a parrot',
 'text': 'Why did the parrot wear a raincoat?\n\nBecause he wanted to be polly unsaturated!'}

In [3]:
prompt_template = PromptTemplate(input_variables=["input", "language"], template="Tell me a joke about {input} in {language}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke({"input": "a parrot", "language": "german"})

{'input': 'a parrot',
 'language': 'german',
 'text': 'Warum hat der Papagei eine Brille auf der Nase?\n\nWeil er Kontaktlinsen nicht vertragen kann!'}

Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [4]:
from langchain.chains import SequentialChain

# This is an LLMChain to write a review given a dish name and the experience.
prompt_review = PromptTemplate.from_template(
    template="You ordered {dish_name} and your experience was {experience}. Write a review: "
)
chain_review = LLMChain(llm=llm, prompt=prompt_review, output_key="review")

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = PromptTemplate.from_template(
    template="Given the restaurant review: {review}, write a follow-up comment: "
)
chain_comment = LLMChain(llm=llm, prompt=prompt_comment, output_key="comment")

# This is an LLMChain to summarize a review.
prompt_summary = PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {comment}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")

# This is an LLMChain to translate a summary into German.
prompt_translation = PromptTemplate.from_template(
    template="Translate the summary to german: \n\n {summary}"
)
chain_translation = LLMChain(
    llm=llm, prompt=prompt_translation, output_key="german_translation"
)

In [5]:
overall_chain = SequentialChain(
    chains=[chain_review, chain_comment, chain_summary, chain_translation],
    input_variables=["dish_name", "experience"],
    output_variables=["review", "comment", "summary", "german_translation"],
)

overall_chain.invoke({"dish_name": "Pizza Salami", "experience": "It was awful!"})

{'dish_name': 'Pizza Salami',
 'experience': 'It was awful!',
 'review': 'I recently ordered the Pizza Salami from this restaurant and I have to say, it was absolutely awful. The crust was burnt and soggy at the same time, the cheese was barely melted, and the salami was dry and tasteless. It was clear that the pizza had been sitting out for far too long before it was delivered to me.\n\nI was extremely disappointed with the quality of the pizza and I definitely will not be ordering from this restaurant again. The whole experience was a letdown and I would not recommend this dish to anyone. Save your money and go somewhere else for a better pizza experience.',
 'comment': "I had a similar experience with the Pizza Salami from this restaurant. I was really looking forward to trying it, but unfortunately, it was not up to par. The burnt and soggy crust, poorly melted cheese, and dry, tasteless salami were all major disappointments.\n\nI agree that it seemed like the pizza had been sittin

Instead of chaining multiple chains together we can also use an LLM to decide which follow up chain is being used

In [6]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

positive_template = """You are an AI that focuses on the positive side of things. \
Whenever you analyze a text, you look for the positive aspects and highlight them. \
Here is the text:
{input}"""

neutral_template = """You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, \
not favoring any positive or negative aspects. Here is the text:
{input}"""

negative_template = """You are an AI that is designed to find the negative aspects in a text. \
You analyze a text and show the potential downsides. Here is the text:
{input}"""

In [7]:
prompt_infos = [
    {
        "name": "positive",
        "description": "Good for analyzing positive sentiments",
        "prompt_template": positive_template,
    },
    {
        "name": "neutral",
        "description": "Good for analyzing neutral sentiments",
        "prompt_template": neutral_template,
    },
    {
        "name": "negative",
        "description": "Good for analyzing negative sentiments",
        "prompt_template": negative_template,
    },
]

In [8]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
destination_chains

{'positive': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='You are an AI that focuses on the positive side of things. Whenever you analyze a text, you look for the positive aspects and highlight them. Here is the text:\n{input}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fb013224310>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fb012ea4450>, openai_api_key=SecretStr('**********'), openai_proxy='')),
 'neutral': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, not favoring any positive or negative aspects. Here is the text:\n{input}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fb013224310>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fb012ea4450>, openai_api_key=SecretStr('**********'), openai_proxy='')),


In [9]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

positive: Good for analyzing positive sentiments
neutral: Good for analyzing neutral sentiments
negative: Good for analyzing negative sentiments


In [10]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=destination_chains["neutral"],
    verbose=True,
)

chain.invoke({"input": "I ordered Pizza Salami for 9.99$ and it was awesome!"})



> Entering new MultiPromptChain chain...
positive: {'input': 'I ordered Pizza Salami for 9.99$ and it was awesome!'}
> Finished chain.


{'input': 'I ordered Pizza Salami for 9.99$ and it was awesome!',
 'text': "I love how you treated yourself to a delicious pizza for a great price! It's wonderful that you enjoyed it so much. Keep indulging in little treats like this to brighten your day."}